# SVM

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxopt
from cvxopt import matrix
from cvxopt import solvers

In [69]:
def ReadInput():
    df = pd.read_csv('./fashion_mnist/train.csv',header=None)
    data = df.to_numpy()
    index = np.logical_or(data[:,784]==4 ,data[:,784]==5)
    X_train = data[index][:,:-1]
    Y_train = np.array([1 if i==4 else -1 for i in data[index][:,-1]]).reshape((-1,1))


    df = pd.read_csv('./fashion_mnist/test.csv',header=None)
    data = df.to_numpy()
    index = np.logical_or(data[:,784]==4 ,data[:,784]==5)
    X_test = data[index][:,:-1]
    Y_test = np.array([1 if i==4 else -1 for i in data[index][:,-1]]).reshape((-1,1))


    df = pd.read_csv('./fashion_mnist/val.csv',header=None)
    data = df.to_numpy()
    index = np.logical_or(data[:,784]==4 ,data[:,784]==5)
    X_val = data[index][:,:-1]
    Y_val = np.array([1 if i==4 else -1 for i in data[index][:,-1]]).reshape((-1,1))
    
    return X_train, Y_train, X_test, Y_test, X_val, Y_val


# PART A 

In [45]:
def LinearSVM_fit(X,Y):
#     X = X_train/255
#     Y = Y_train
    G1 = np.diag(np.array([1]*Y.shape[0]))
    G2 = np.diag(np.array([-1]*Y.shape[0]))
    G = matrix(np.append(G1,G2,axis=0),tc='d')
    c = 1
    H = matrix(np.append(np.array([c]*Y.shape[0]),np.array([0]*Y.shape[0]),axis = 0),tc='d')
    P = matrix(np.dot(X,X.T)*np.dot(Y,Y.T),tc='d')
    Q = matrix(np.array([-1]*Y.shape[0]).T,tc='d')
    A = matrix(Y.T,tc='d')
    B = matrix(np.array(0).reshape((1,1)),tc='d')

    sol = solvers.qp(P,Q,G,H,A,B)

    alpha=np.array(sol['x'])
    
    #calculating the support vectors given a threshold
    threshold = 1e-15
    data = np.append(X,Y,axis = 1)
    data_with_alpha = np.append(data,alpha,axis = 1)
    
    supportVectors = data_with_alpha[np.where(data_with_alpha[:,-1] >= threshold)]
    print("Total supports vectors")
    print(supportVectors.shape)
    print('\n')
    # w = SUM( x*y*alpha )
    w = np.sum(supportVectors[:,:-2]*supportVectors[:,-2].reshape((-1,1))*supportVectors[:,-1].reshape((-1,1)),axis=0).reshape((-1,1))

    pos_X = supportVectors[np.where(supportVectors[:,-2]==1)][:,:-2]
    neg_X = supportVectors[np.where(supportVectors[:,-2]==-1)][:,:-2]
    
    b = -(np.dot(pos_X,w).min() + np.dot(neg_X,w).max())/2
    return w,b,supportVectors

In [67]:
def LinearSVM_predict(X,Y,w,b):
    pred = []
    for x in X:
        if (np.dot(w.T,x)+b) > 0:
            pred.append(1)
        else:
            pred.append(-1)

    count =0

    for i in range(len(pred)):
        if pred[i] == Y[i]:
            count+=1
    print("Accurarcy is ",count*100/Y.shape[0]," %") 

<font color=red>DATA INPUT</font>

In [47]:
X_train, Y_train, X_test, Y_test, X_val, Y_val = ReadInput()

<font color=red>LEARNING</font>

In [62]:

w,b,supportVectors = LinearSVM_fit(X_train/255,Y_train)

     pcost       dcost       gap    pres   dres
 0: -1.8144e+02 -7.2340e+03  4e+04  2e+00  1e-12
 1: -1.0203e+02 -3.4492e+03  6e+03  3e-01  1e-12
 2: -3.3534e+01 -9.1974e+02  2e+03  7e-02  6e-13
 3: -1.1925e+01 -4.1859e+02  7e+02  2e-02  3e-13
 4: -3.6847e+00 -1.7770e+02  3e+02  8e-03  1e-13
 5: -1.2594e+00 -5.9907e+01  9e+01  2e-03  5e-14
 6: -6.7211e-01 -2.5705e+01  4e+01  9e-04  3e-14
 7: -4.1327e-01 -1.1925e+01  2e+01  3e-04  2e-14
 8: -5.1263e-01 -5.5495e+00  6e+00  1e-04  1e-14
 9: -5.7700e-01 -3.1645e+00  3e+00  4e-05  1e-14
10: -7.0634e-01 -2.0084e+00  1e+00  3e-06  2e-14
11: -9.3593e-01 -1.4177e+00  5e-01  7e-07  2e-14
12: -1.0291e+00 -1.2146e+00  2e-01  4e-16  2e-14
13: -1.0863e+00 -1.1319e+00  5e-02  4e-16  2e-14
14: -1.1048e+00 -1.1100e+00  5e-03  5e-16  2e-14
15: -1.1073e+00 -1.1074e+00  1e-04  5e-16  2e-14
16: -1.1073e+00 -1.1073e+00  1e-06  6e-16  2e-14
17: -1.1073e+00 -1.1073e+00  1e-08  5e-16  2e-14
Optimal solution found.
Total supports vectors
(4500, 786)




<font color=red>PREDICTING</font>

In [68]:
LinearSVM_predict(X_test/255,Y_test,w,b)
LinearSVM_predict(X_val/255,Y_val,w,b)
LinearSVM_predict(X_train/255,Y_train,w,b)

Accurarcy is  99.8  %
Accurarcy is  99.6  %
Accurarcy is  100.0  %
